# Example 8 -- MODFLOW Application Programming Interface (API)

## Create a Simple Model

In [ ]:
import pathlib as pl
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon, LineString
import flopy

ws = './ex8'
name = 'mymodel'
tempdir = pl.Path(ws, "temp")
tempdir.mkdir(parents=True, exist_ok=True)

domain = Polygon([
    [1831.38, 6335.54],
    [4337.73, 6851.13],
    [6428.74, 6707.91],
    [8662.98, 6493.08],
    [9350.43, 5891.56],
    [9235.86, 4717.15],
    [8963.74, 3685.97],
    [8691.62, 2783.68],
    [8047.13, 2038.94],
    [7416.96, 578.09],
    [6414.42, 105.46],
    [5354.59, 205.72],
    [4624.17, 363.26],
    [3363.83, 563.77],
    [1330.11, 1809.78],
    [399.18, 2998.51],
    [914.77, 5132.49],
])

Lx = 10000
Ly = 8000
nlay = 1
nrow = 32
ncol = 40
delr = Lx / ncol * np.ones(ncol, dtype=float)
delc = Ly / nrow * np.ones(nrow, dtype=float)
top = 50 * np.ones((nrow, ncol), dtype=float)
botm = -100 * np.ones((nlay, nrow, ncol), dtype=float)
sg = flopy.discretization.StructuredGrid(
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm
)
idomain = np.zeros((nlay, nrow, ncol), dtype=int)
gi = flopy.utils.GridIntersect(sg)

# inside domain polygon
ixp = gi.intersect(domain)
for i, j in ixp["cellids"]:
    idomain[:, i, j] = 1

# identify cells that touch domain polygon
ls = LineString([p for p in domain.exterior.coords])    
ixl = gi.intersect(ls)
for i, j in ixl["cellids"]:
    idomain[:, i, j] = 2

sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=ws, exe_name='mf6')
tdis = flopy.mf6.ModflowTdis(sim)
ims = flopy.mf6.ModflowIms(sim, print_option="all", inner_maximum=100)
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True)
dis = flopy.mf6.ModflowGwfdis(
    gwf, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, 
    top=top, botm=botm, idomain=idomain)
ic = flopy.mf6.ModflowGwfic(gwf, strt=top)
npf = flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True, icelltype=1)
chdspd = [[(0, i, j), 1.] for i, j in ixl["cellids"]]
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chdspd)
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=0.001)
oc = flopy.mf6.ModflowGwfoc(gwf,
                            budget_filerecord=f"{name}.bud",
                            head_filerecord=f"{name}.hds",
                            printrecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
                            saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')])
sim.write_simulation()

## Run Model from Python Using the API

### Control and Run by Time Step

In [ ]:
import modflowapi
libmf6 = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/libmf6.dylib"

mf6 =  modflowapi.ModflowApi(libmf6, working_directory=ws)
print("Initializing mf6...")
mf6.initialize()
current_time = 0.
end_time = mf6.get_end_time()

while current_time < end_time:
    print("  Updating mf6...")
    mf6.update()
    current_time = mf6.get_current_time()

print ("Finalizing mf6...")
mf6.finalize()

### Control and Run by Iteration

In [ ]:
import modflowapi
libmf6 = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/libmf6.dylib"

mf6 =  modflowapi.ModflowApi(libmf6, working_directory=ws)
print("Initializing mf6...")
mf6.initialize()
current_time = 0.
end_time = mf6.get_end_time()

while current_time < end_time:
    
    print("  Get and prepare time step...")
    dt = mf6.get_time_step()
    mf6.prepare_time_step(dt)
    
    kiter = 0
    print("  Prepare solve...")
    mf6.prepare_solve(1)
    while kiter < 30:
        print("    Solve...")
        has_converged = mf6.solve(1)
        if has_converged:
            break
    print("  Finalize solve...")
    mf6.finalize_solve(1)
    
    print("  Finalize time step...")
    mf6.finalize_time_step()
    current_time = mf6.get_current_time()

print ("Finalizing mf6...")
mf6.finalize()

### Print Head Min and Max Between Iteration

In [ ]:
import modflowapi
libmf6 = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/libmf6.dylib"

mf6 =  modflowapi.ModflowApi(libmf6, working_directory=ws)
print("Initializing mf6...")
mf6.initialize()

# get a pointer to the head variable
head = mf6.get_value_ptr("SLN_1/X")

current_time = 0.
end_time = mf6.get_end_time()

while current_time < end_time:
    
    print("  Get and prepare time step...")
    dt = mf6.get_time_step()
    mf6.prepare_time_step(dt)
    
    kiter = 0
    print("  Prepare solve...")
    mf6.prepare_solve(1)
    while kiter < 30:
        print("    Solve...")
        has_converged = mf6.solve(1)
        print (f"    Head min ({head.min()}) max ({head.max()})")
        if has_converged:
            break
    print("  Finalize solve...")
    mf6.finalize_solve(1)
    
    print("  Finalize time step...")
    mf6.finalize_time_step()
    current_time = mf6.get_current_time()

print ("Finalizing mf6...")
mf6.finalize()